In [1]:
import whisper
import ffmpeg
import numpy as np
import torch
import pyaudio

# Get input audio

# Process audio and turn to TEXT

# TEXT to SPEECH

In [3]:
def pyaudio_get_output_device(searching_for = 'Logitech'):
    # Input params
    # Searching for - partial name of device which should be used. UPPER AND LOWER CASE MATTERS ! 

    # Find device
    good_device_info = None
    aud = pyaudio.PyAudio()
    for i in range(0,15):
        try:
            info = aud.get_device_info_by_index(i)
            #print(info['name'])
            if searching_for in info['name']:
                if info['maxOutputChannels']>=1:
                    print(info)
                    good_device_info = info
        except Exception as e :
            print("Exception:", e)
            
    aud.terminate()
    if good_device_info is None:
        raise Exception("Did not find a good input device !")
    else:
        return good_device_info

info = pyaudio_get_output_device()
good_idx = info

{'index': 8, 'structVersion': 2, 'name': 'Logitech USB Headset: Audio (hw:3,0)', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.008, 'defaultLowOutputLatency': 0.008, 'defaultHighInputLatency': 0.032, 'defaultHighOutputLatency': 0.032, 'defaultSampleRate': 48000.0}
Exception: [Errno -9996] Invalid device info
Exception: [Errno -9996] Invalid device info
Exception: [Errno -9996] Invalid device info
Exception: [Errno -9996] Invalid device info
Exception: [Errno -9996] Invalid device info
Exception: [Errno -9996] Invalid device info


ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave


In [4]:
from TTS.api import TTS

class TextToSpeech:
    def __init__(self):
        # Running a multi-speaker and multi-lingual model

        # List available 
        model_name = TTS.list_models()[0]
        # Init TTS
        self.tts = TTS(model_name, progress_bar = False, gpu = True);
    def text_to_speech(self, text):
        self.text_to_speech_to_file(text, output = 'tmp1.wav')
        
        
    def text_to_speech_to_file(self, text, output = 'output.wav'):
        # Run TTS
        self.tts.tts_to_file(text=text, speaker=self.tts.speakers[4], language=self.tts.languages[0], 
                             file_path="output.wav")
        #wav = tts.tts("This is a test! This is also a test!!", speaker=tts.speakers[1], language=tts.languages[0])
        # Text to speech to a file
        #tts.tts_to_file(text="Hello world!", speaker=tts.speakers[0], language=tts.languages[0], file_path="output.wav")
txt2spch = TextToSpeech()

 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--your_tts
 > Model's license - CC BY-NC-ND 4.0
 > Check https://creativecommons.org/licenses/by-nc-nd/4.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 

In [34]:
txt2spch.text_to_speech_to_file(text = 'let me tell you a little something something')

 > Text splitted to sentences.
['let me tell you a little something something']
 > Processing time: 0.05411958694458008
 > Real-time factor: 0.02181361827673522


# ChatGPT API

In [4]:
# TODO - Get a ChatGPT API which actually works.

# Streaming audio

In [9]:
from audio_utils import record_clip, pyaudio_get_input_mic_device, SpeechToTextHandler
# Get good device info
device_info = pyaudio_get_input_mic_device()
# Initialize VAD and stuffs

device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("base").to(device)

Speech_to_text_obj = SpeechToTextHandler(nospinner = False, model=model, savewav = False, vad_aggressiveness=0,
                                         device = device_info['index'], rate = int(device_info['defaultSampleRate']),file = 'out.wav')

{'index': 8, 'structVersion': 2, 'name': 'Logitech USB Headset: Audio (hw:3,0)', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.008, 'defaultLowOutputLatency': 0.008, 'defaultHighInputLatency': 0.032, 'defaultHighOutputLatency': 0.032, 'defaultSampleRate': 48000.0}
Exception: [Errno -9996] Invalid device info
Exception: [Errno -9996] Invalid device info
Exception: [Errno -9996] Invalid device info
Exception: [Errno -9996] Invalid device info
Exception: [Errno -9996] Invalid device info
Exception: [Errno -9996] Invalid device info


ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave


KWARGS: {'format': 8, 'channels': 1, 'rate': 48000, 'input': True, 'frames_per_buffer': 960, 'stream_callback': <function Audio.__init__.<locals>.proxy_callback at 0x7f3a8440f700>, 'input_device_index': 8}


ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave


In [10]:
Speech_to_text_obj.stream_from_mic()

Listening (ctrl-C to exit)...
Thank you.
Tell me something.
Thank you.\ | / - \ | 
Thank you.\ | / - \ | / - \ | / - \ | / - \ | 
Don't act late. - \ | / 
Don't act late. - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 
4 to 4, go to the audio.- \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 
you | / 
What's the mod? Pick a mod. | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 
Thank you.\ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 
3 is the most aggressive- \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 
- \ | 